In [ ]:
# 重定向修复：将已下载的html网页中（被站内“包装”隐藏起来）的链接解开一层重定向，替换为重定向后的地址，以适应资源库无法访问的情况，避免失联

In [ ]:
import html
import requests
import time
from sine.utils import *
from multiprocessing.pool import ThreadPool

In [ ]:
output_file = 'D:\\Downloads\\Jay的资源库 - 公众号：无与伦比的Jay同学2.html'
with open('D:\\Downloads\\Jay的资源库 - 公众号：无与伦比的Jay同学.html', 'r', encoding='utf-8') as f:
    txt = f.read()

In [ ]:
def isUrlNeedDeal(oldhref):
    '''重定向链接初步筛选。本站定制'''
    return oldhref.find('index.php') > 0

In [ ]:
# 以本文形式处理，识别出链接元素及其后的url地址，后续做替换
split_str='<a href="'
group = txt.split(split_str)

In [ ]:
HEADERS = {
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-CN,zh;q=0.9',
    'pragma': 'no-cache',
    'cache-control': 'no-cache',
    'upgrade-insecure-requests': '1',
    'user-agent': "Mozilla/5.0 (iPhone; CPU iPhone OS 11_0 like Mac OS X) AppleWebKit/604.1.38 (KHTML, like Gecko) Version/11.0 Mobile/15A372 Safari/604.1",
}
def get_real_address(url):
    '''处理302重定向请求'''
    res = requests.get(url, headers=HEADERS, allow_redirects=False)
    return res.headers['Location'] if res.status_code == 302 else None

In [ ]:
hrefs = []
for i in range(1, len(group)):
    href = group[i].split('"', 1)
    hrefs.append(href[0])

In [ ]:
hsize = len(hrefs)
bar = PredictableProcessBar(total=hsize, unit='个', smoothing=0)
def download_302(i, bar):
    oldgroup = group[i + 1]
    oldhref = hrefs[i]
    if isUrlNeedDeal(oldhref):
        newhref = get_real_address(html.unescape(oldhref))
        if newhref != None:
            group[i + 1] = oldgroup.replace(oldhref, newhref)
            hrefs[i] = newhref
    bar.update(1)
with ThreadPool(20) as executor, PredictableProcessBar(total=hsize) as bar:
    for i in range(hsize):
        executor.apply_async(download_302, args=(i, bar), callback=None)
    executor.close()
    executor.join()
    bar.display()
    time.sleep(1)
    bar.display()

In [ ]:
newtxt = split_str.join(group)

In [ ]:
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(newtxt)